In [ ]:
# Conditioning on the particle type, masking the velocity field

In [1]:
!pip install --user torchcfm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 23.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 11.4 MB/s eta 0:00:00
DEPRECATION: gosam 2.1.1-4b98559 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of gosam or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [6]:
!python3.11 -m pip install --user torchdyn

ERROR: Can not combine '--user' and '--target'


In [7]:
!python3.11 -m pip show torchdyn

In [9]:
import sys
python311_user_path = '/eos/user/m/mmcohen/.local/lib/python3.11/site-packages'
if python311_user_path not in sys.path:
    sys.path.insert(0, python311_user_path)

print("Updated sys.path:")
print(sys.path)

# Now try importing
try:
    from torchdyn.core import NeuralODE
    print("Success! torchdyn imported successfully")
except ImportError as e:
    print(f"Still can't import: {e}")

Updated sys.path:
['/eos/user/m/mmcohen/.local/lib/python3.11/site-packages', '/eos/home-i03/m/mmcohen/ctfm_development/src/CTFM_AD/event_level_AD', '', '/eos/user/m/mmcohen/.local/lib/python3.9/site-packages', '/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/itk', '/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/python', '/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib', '/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages', '/usr/local/lib/swan/nb_term_lib', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc11-opt/lib/python311.zip', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc11-opt/lib/python3.11', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc11-opt/lib/python3.11/lib-dynload', '/cvmfs/sft.cern.ch/lcg/releases/Python/3.11.9-2924c/x86_64-el9-gcc11-opt/lib/python3.11/site-packages', '/cvmfs/sft.cern.ch/

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
#from torchdyn.core import NeuralODE
from torchcfm.conditional_flow_matching import ExactOptimalTransportConditionalFlowMatcher
from matplotlib import animation
import numpy as np
import load_and_preprocess as lap
import train_and_eval_functions as taef
import os
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

2025-05-19 13:54:26.328356: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
import torch
import sys

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"PyTorch file location: {torch.__file__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")

Python version: 3.11.9 (main, Jun 24 2024, 14:32:54) [GCC 11.3.0]
PyTorch version: 2.3.1
PyTorch file location: /cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/torch/__init__.py
CUDA available: True
CUDA version: 12.3


In [18]:
print(f"GPU device: {torch.cuda.get_device_name(0)}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
print(f"GPU compute capability: {torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}")

# Test a simple CUDA operation
try:
    x = torch.randn(10, 10).cuda()
    y = torch.randn(10, 10).cuda()
    z = torch.mm(x, y)
    print("Basic CUDA operations work fine")
    print(f"Result shape: {z.shape}")
except Exception as e:
    print(f"Basic CUDA test failed: {e}")

GPU device: NVIDIA A100-PCIE-40GB
GPU count: 1
Current device: 0
GPU memory: 42.5 GB
GPU compute capability: 8.0
Basic CUDA operations work fine
Result shape: torch.Size([10, 10])


In [11]:
label = 'otfm_40mhz_sigma0p01'

batchSize = 512
numberOfEpochs = 20
patience = 5

pxpypz = True
p_train = 0.5
p_test = 0.25
plots_path = '/eos/home-m/mmcohen/ctfm_development/trained_models/trial_1/plots'

input_dim = 3
model_dim = 128
ff_dim = 128
num_heads = 4
num_layers = 4
n_mask_vals = 5 # 4 particle types + 1 padding

flowSigma = 0.01

trained = False
multiGPU = True


if not multiGPU:
    os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [12]:
datasets = lap.load_and_preprocess(
    p_train=p_train,
    p_test=p_test,
    plots_path=plots_path,
    pxpypz=pxpypz
    )

Extracting...
Loaded bkg
Loaded a4l
Loaded htaunu
Loaded htautau
Loaded lq
a4l: (55969, 19, 4)
htaunu: (760272, 19, 4)
htautau: (691283, 19, 4)
lq: (340544, 19, 4)
bkg_train: (1000000, 19, 4)
bkg_test: (1000000, 19, 4)
bkg_val: (1000000, 19, 4)


In [13]:
class TransformerVectorField(nn.Module):
    def __init__(self, input_dim=3, model_dim=128, num_heads=8, num_layers=4, 
                 n_mask_vals=5, ff_dim=512):
        super().__init__()
        self.model_dim = model_dim

        # project the 3D features
        self.input_proj = nn.Linear(input_dim, model_dim)

        # embed mask (0=pad, 1-4=particle types)
        self.mask_emb = nn.Embedding(n_mask_vals, model_dim)

        # time embedding MLP
        self.time_mlp = nn.Sequential(
            nn.Linear(1, model_dim),
            nn.SiLU(),
            nn.Linear(model_dim, model_dim)
        )

        # transformer stack
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=model_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            activation='gelu',
            batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # back to velocity
        self.output_proj = nn.Linear(model_dim, input_dim)

    def forward(self, t, x, mask=None, **kwargs):
        # x: [B, N, 3], mask: [B, N, 1]
        B, N, _ = x.shape

        # 1) feature projection
        h = self.input_proj(x)                          # [B, N, model_dim]

        # 2) mask embedding and injection
        if mask is not None:
            m = mask.squeeze(-1).long()                        # [B, N]
            m_emb = self.mask_emb(m)                    # [B, N, model_dim]
            h = h + m_emb

        # 3) time embedding
        if isinstance(t, float) or t.ndim == 0:
            t = torch.full((B, 1), float(t), device=x.device)
        elif t.ndim == 1:
            t = t.view(-1, 1)
        time_emb = self.time_mlp(t)[:, None, :]         # [B, 1, model_dim]
        h = h + time_emb                                # broadcast to [B, N, model_dim]

        # 4) transformer with padding mask
        src_key_padding_mask = None
        if mask is not None:
            # mask==0 indicates padding positions
            src_key_padding_mask = (mask.squeeze(-1) == 0)  # [B, N] boolean
        h = self.transformer(h, src_key_padding_mask=src_key_padding_mask)

        # 5) output velocity
        v = self.output_proj(h)                         # [B, N, 3]
        # zero out velocities on padded slots
        if mask is not None:
            binary_mask = (mask > 0).float()   # [B, N, 1]: 1 if particle exists, 0 if padded
            v = v * binary_mask
        return v

In [14]:
def train(model, dataloader, optimizer, device, num_epochs=50, sigma=0.0):
    model.to(device)
    # flow matcher does not support conditioning directly; we embed and mask slots instead
    FM = ExactOptimalTransportConditionalFlowMatcher(sigma=sigma)

    for epoch in range(num_epochs):
        total_loss = 0.
        for i, (x1, mask) in enumerate(tqdm(dataloader)):
            x1 = x1.to(device)
            mask = mask.to(device)
            optimizer.zero_grad()

            # sample noise
            x0 = torch.randn_like(x1)
            # sample flow-matching tuples with condition
            t, xt, ut = FM.sample_location_and_conditional_flow(x0, x1)

            # model prediction
            vt = model(t, xt, mask=mask)
            # mask ground-truth velocity
            binary_mask = (mask > 0).float()            # [B, N, 1]
            ut = ut * binary_mask

            # compute loss only on active slots
            # vt, ut: [B, N, D], binary_mask: [B, N, 1]
            mask_expanded = binary_mask.expand_as(vt)            # [B, N, D]
            sq_err = (vt - ut)**2                                # [B, N, D]
            loss = torch.sum(sq_err * mask_expanded)             # sum over only active dims
            loss = loss / mask_expanded.sum().clamp(min=1)  
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1} - Loss: {avg_loss:.6f}")

In [15]:
class EvtDataset(Dataset):
    def __init__(self, evts):  # jets: [N, maxparts, 4]
        self.x = evts[:, :, :3]           # [eta, phi, pt]
        self.mask = evts[:, :, 3:]        # [mask]

    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        return self.x[idx], self.mask[idx]

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = EvtDataset(datasets['bkg_train']['data'].astype(np.float32))
dataloader = DataLoader(dataset, batch_size=batchSize, shuffle=True, drop_last=True)

model = TransformerVectorField(input_dim=input_dim, model_dim=model_dim, num_heads=num_heads, num_layers=num_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

if not trained:
    train(model, dataloader, optimizer, device=device, num_epochs=numberOfEpochs, sigma=flowSigma)
    
    # Assuming `node` is your trained NeuralODE (e.g., torchcfm.NeuralODE)
    torch.save({
        'state_dict': model.state_dict(),
        'model_kwargs': {
            'input_dim': input_dim,
            'model_dim': model_dim,
            'num_heads': num_heads,
            'num_layers': num_layers
        }
    }, "flow_model_%s.pt"%label)

else:
    # Rebuild the vector field
    ckpt = torch.load("flow_model_%s.pt"%label, map_location=device)
    
    # Recreate the architecture
    model = TransformerVectorField(**ckpt['model_kwargs'])
    model.load_state_dict(ckpt['state_dict'])
    model.to(device).eval()

node = NeuralODE(model, solver="dopri5", sensitivity="adjoint", atol=1e-4, rtol=1e-4)

  0%|          | 0/1953 [00:00<?, ?it/s]


RuntimeError: CUDA error: no kernel image is available for execution on the device
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# animate flows

In [ ]:
def map_to_base_distribution(node, x1, mask, batch_size=1024):
    """
    Maps x1 (data space) to x0 (base space) using the inverse flow in batches.
    Applies mask to ignore padded particles.
    
    Args:
        node: A trained NeuralODE.
        x1: [B, N, D] torch.Tensor - data space samples.
        mask: [B, N, 1] torch.Tensor - binary mask for valid particles.
        batch_size: int - batch size for processing.

    Returns:
        x0: [B, N, D] torch.Tensor - mapped base samples with mask applied.
    """
    t_span = torch.linspace(1., 0., 2, device=x1.device)
    outputs = []

    for i in tqdm(range(0, x1.size(0), batch_size)):
        xb = x1[i:i+batch_size]
        mb = mask[i:i+batch_size]
        binary_mask = (mb > 0).float()

        with torch.no_grad():
            traj = node.trajectory(xb, t_span=t_span)  # [2, bsz, N, D]
            x0_b = traj[-1]  * binary_mask

        outputs.append(x0_b)

    return torch.cat(outputs, dim=0)

In [ ]:
skip_tags = ['bkg_val', 'bkg_train']
for tag, data_dict in datasets.items():
    if tag in skip_tags:
        continue
    
    torch_data = torch.tensor(data_dict['data'], dtype=torch.float32).to(device)
    torch_x = torch_data[:,:,:3]
    torch_mask = torch_data[:,:,3:]

    data_dict['x0_latent'] = map_to_base_distribution(node, torch_x, torch_mask)
    data_dict['torch_mask'] = torch_mask

    print(tag)
    print("Mapped shape:", data_dict['x0_latent'].shape)  # Should be [B, maxparts, 3]
    print("Latent mean (should be ~0):", data_dict['x0_latent'].mean().item())
    print("Latent std (should be ~1):", data_dict['x0_latent'].std().item())

In [ ]:
# plot features post flow
plt.figure(figsize=(12, 8))
plt.hist(datasets['bkg_test']['x0_latent'][:,:,0].flatten()[datasets['bkg_test']['x0_latent'][:,:,-1].flatten()>0.5], bins=50, histtype='step', density=True, label='feature 1')
plt.hist(datasets['bkg_test']['x0_latent'][:,:,1].flatten()[datasets['bkg_test']['x0_latent'][:,:,-1].flatten()>0.5], bins=50, histtype='step', density=True, label='feature 2')
plt.hist(datasets['bkg_test']['x0_latent'][:,:,2].flatten()[datasets['bkg_test']['x0_latent'][:,:,-1].flatten()>0.5], bins=50, histtype='step', density=True, label='feature 3')
plt.yscale('log')
plt.legend()
plt.savefig(f'{plots_path}/post_flow_feature_histograms.png')
plt.close()

In [ ]:
def neg_log_prob_gaussian(x0, mask):
    """
    Compute -log p(x0) under a standard N-dimensional Gaussian,
    masking out invalid (padded) particles.

    Args:
        x0: [B, N, D] tensor
        mask: [B, N, 1] binary mask tensor (1 = valid, 0 = padded)

    Returns:
        nll: [B] tensor, negative log-likelihood per jet
    """
    # Standard Gaussian: log p(x) = -0.5 * (x^2 + log(2π))
    log_probs = -0.5 * (x0 ** 2 + torch.log(torch.tensor(2 * torch.pi, device=x0.device)))

    # Apply mask and sum over particles and features
    binary_mask = (mask > 0).float()
    log_probs_masked = log_probs * binary_mask  # [B, N, D]
    log_likelihood = log_probs_masked.sum(dim=(1, 2))  # [B]

    # Return negative log-likelihood
    return -log_likelihood  # [B]

In [ ]:
for tag, data_dict in datasets.items():
    if tag in skip_tags:
        continue
    data_dict['AD_scores'] = neg_log_prob_gaussian(data_dict['x0_latent'], data_dict['torch_mask']).cpu().numpy()  # [B]
    print(tag)
    print("Mean NLL per jet:", np.mean(data_dict['AD_scores']))

In [ ]:
# Plot AD scores
plt.figure(figsize=(12, 8))
for tag, data_dict in datasets.items():
    if tag in skip_tags:
        continue
    plt.hist(data_dict['AD_scores'], bins=50, histtype='step', density=True, label=tag)

plt.yscale('log')
plt.legend()
plt.savefig(f'{plots_path}/AD_scores_histograms.png')
plt.close()

In [ ]:
# ROC curve
plt.figure(figsize=(10, 8))
    
bkg_scores = datasets['bkg_test']['AD_scores']

for tag, data_dict in datasets.items():
    if 'bkg' in tag:
        continue

    sig_scores = data_dict['AD_scores']
    combined_scores = np.concatenate([bkg_scores, sig_scores], axis=0)
    combined_labels = np.concatenate([np.zeros_like(bkg_scores), np.ones_like(sig_scores)], axis=0)

    fpr, tpr, thresholds = roc_curve(combined_labels, combined_scores)
    auroc = auc(fpr, tpr)

    plt.plot(fpr, tpr, label=f'{tag} (auc = {auroc:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random')
plt.annotate('better', xy=(0.7, 0.3), xytext=(0.5, 0.1),
                textcoords='axes fraction', fontsize=12, color='red',
                arrowprops=dict(facecolor='red', shrink=0.05, width=2, headwidth=8))

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.yscale('log')
plt.xscale('log')
plt.legend()
plt.savefig(f'{plots_path}/roc_curves.png')
plt.close()